In [1]:
import subprocess

# Function to install a package if it's not already installed
def install_package(package_name):
    try:
        __import__(package_name)
    except ImportError:
        subprocess.check_call(['pip', 'install', package_name])

# Ensure necessary packages are installed
install_package('pandas')
install_package('requests')
install_package('pyperclip')
install_package('os')
install_package('io')
install_package('sqlite3')

# Now import the packages
import requests
import pandas as pd
from io import StringIO
import sqlite3
import os
import pyperclip

In [2]:
# URLs for Google Sheets
url_track_streams = 'https://docs.google.com/spreadsheets/d/1t7EUQ6zAd-Y5kDK7qbyxmJzYX6CwJUdpG9_I8-nAYhs/export?format=csv&gid=149563190'
url_track_artists = 'https://docs.google.com/spreadsheets/d/1t7EUQ6zAd-Y5kDK7qbyxmJzYX6CwJUdpG9_I8-nAYhs/export?format=csv&gid=1500032516'
url_track_names = 'https://docs.google.com/spreadsheets/d/1t7EUQ6zAd-Y5kDK7qbyxmJzYX6CwJUdpG9_I8-nAYhs/export?format=csv&gid=1258211082'

# Step 1: Fetch data from Google Sheets
response = requests.get(url_track_streams)
data = response.content
df_track_streams = pd.read_csv(StringIO(data.decode('utf-8')))
df_track_streams['streams'] = pd.to_numeric(df_track_streams['streams'], errors='coerce')

response = requests.get(url_track_artists)
data = response.content
df_track_artists = pd.read_csv(StringIO(data.decode('utf-8')))

response = requests.get(url_track_names)
data = response.content
df_track_names = pd.read_csv(StringIO(data.decode('utf-8')))

# Step 2: Create a SQLite database
db_name = 'spotify_database.db'
conn = sqlite3.connect(db_name)
cur = conn.cursor()



In [3]:
# Create table and insert data for track_streams
columns = df_track_streams.columns
columns_str = ', '.join([f'"{col}" TEXT' for col in columns])
create_table_query = f'CREATE TABLE IF NOT EXISTS track_streams ({columns_str});'
cur.execute(create_table_query)
df_track_streams.to_sql('track_streams', conn, if_exists='replace', index=False)

# Create table and insert data for track_artists
columns = df_track_artists.columns
columns_str = ', '.join([f'"{col}" TEXT' for col in columns])
create_table_query = f'CREATE TABLE IF NOT EXISTS track_artists ({columns_str});'
cur.execute(create_table_query)
df_track_artists.to_sql('track_artists', conn, if_exists='replace', index=False)

# Create table and insert data for track_names
columns = df_track_names.columns
columns_str = ', '.join([f'"{col}" TEXT' for col in columns])
create_table_query = f'CREATE TABLE IF NOT EXISTS track_names ({columns_str});'
cur.execute(create_table_query)
df_track_names.to_sql('track_names', conn, if_exists='replace', index=False)

1277

In [4]:
# Commit the changes
conn.commit()

# Step 3: Print the file path to the SQLite database and copy to clipboard
db_path = os.path.abspath(db_name)
print(f"SQLite database file path: {db_path}")

# Copy the file path to clipboard
pyperclip.copy(db_path)
print("Database file path copied to clipboard.")

# Close the connection
conn.close()

SQLite database file path: /Users/kevinbradley/spotify_streams.db
Database file path copied to clipboard.
